In [15]:
from PIL import Image
from torchvision import transforms
import timm
import torch
import torch.nn as nn

# Load model from Hugging Face Hub
model = timm.create_model("hf_hub:Snarcy/RedDino-base", pretrained=True)
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load and preprocess image
image = Image.open("C:/Code/DL/bbosis/Hongyuan-Babesiosis/data/core_data/2.jpg").convert("RGB")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
input_tensor = transform(image).unsqueeze(0).to(device)

# Extract features
with torch.no_grad():
    embedding = model(input_tensor)


In [13]:
embedding
embedding.size()

torch.Size([1, 768])

In [16]:
class ClassifierHead(nn.Module):
    def __init__(self, base_model, num_classes):
        super().__init__()
        self.base_model = base_model
        self.classifier = nn.Linear(768, num_classes)  

    def forward(self, x):
        features = self.base_model.forward_features(x)
        logits = self.classifier(features)
        return logits
    
num_classes = 2
m_model = ClassifierHead(model, num_classes).to(device)